sbpy.data.Ephem Example Notebooks
=================================

[Ephem](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Ephem.html#sbpy.data.Ephem) provides functionality to query, calculate, manipulate, and store ephemerides and observational information.

Querying Asteroid Ephemerides from JPL Horizons
---------------------------------------

Query ephemerides for three asteroids in a given date range as observed from Maunakea (observatory code 568) using [sbpy.data.Ephem.from_horizons](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Ephem.html#sbpy.data.Ephem.from_horizons). This function uses [astroquery.jplhorizons](https://astroquery.readthedocs.io/en/latest/jplhorizons/jplhorizons.html) in the background and create Ephem objects from the query results:

In [1]:
from sbpy.data import Ephem

targets = ['2100', '2018 RC1', 'Ganymed']
eph = Ephem.from_horizons(targets, location=568, 
                          epochs={'start': '2018-01-01', 'stop': '2018-02-01', 'step':'1h'})

print(eph.table)

       targetname           datetime_str   ...  PABLon   PABLat 
                                           ...   deg      deg   
------------------------ ----------------- ... -------- --------
2100 Ra-Shalom (1978 RA) 2018-Jan-01 00:00 ... 305.4411   7.5319
2100 Ra-Shalom (1978 RA) 2018-Jan-01 01:00 ...  305.475   7.5269
2100 Ra-Shalom (1978 RA) 2018-Jan-01 02:00 ...  305.509    7.522
2100 Ra-Shalom (1978 RA) 2018-Jan-01 03:00 ... 305.5431    7.517
2100 Ra-Shalom (1978 RA) 2018-Jan-01 04:00 ... 305.5771   7.5121
2100 Ra-Shalom (1978 RA) 2018-Jan-01 05:00 ... 305.6111   7.5071
2100 Ra-Shalom (1978 RA) 2018-Jan-01 06:00 ... 305.6452   7.5022
2100 Ra-Shalom (1978 RA) 2018-Jan-01 07:00 ... 305.6793   7.4972
2100 Ra-Shalom (1978 RA) 2018-Jan-01 08:00 ... 305.7135   7.4922
2100 Ra-Shalom (1978 RA) 2018-Jan-01 09:00 ... 305.7476   7.4872
                     ...               ... ...      ...      ...
  1036 Ganymed (1924 TD) 2018-Jan-31 15:00 ...  173.719 -22.4957
  1036 Ganymed (1924 TD) 

Querying Satellite Ephemerides from JPL Horizons (and other ambiguous queries)
------------------------------------------------

Query ephemerides for satellite `'GOES-1'` for a number of epochs as observed from Magdalena Ridge Observatory (observatory code `'H01'`):

In [2]:
eph = Ephem.from_horizons('GOES-1', epochs=[2453452.123245, 2453453.34342, 2453454.342342])

ValueError: Unknown target (GOES-1). Maybe try different id_type?

This function call raises a `ValueError`. The reason for this error is that astroquery.jplhorizons assumes that the provided target string is a small body. However, GOES-1 is an artifical satellite. As suggested in the provided error message, we need to specificy an `id_type` that is different from the default value (`'smallbody'`). Since sbpy.data.Ephem.from_horizons builds upon astroquery.jplhorizons, we can provide optional parameters for the latter directly to the former. According to the [JPL Horizons documentation](https://ssd.jpl.nasa.gov/?horizons_doc#selection), spacecraft are grouped together with major body, so let's try:

In [3]:
eph = Ephem.from_horizons('GOES-1', id_type='majorbody', epochs=[2453452.123245, 2453453.34342, 2453454.342342])

ValueError: Ambiguous target name; provide unique id:
  ID#      Name                               Designation  IAU/aliases/other   
  -------  ---------------------------------- -----------  ------------------- 
  -108366  GOES-1 (spacecraft)                                                  
  -135491  GOES-14 (spacecraft)                                                 
 

Another error. This time, the error referes to the target name being ambiguous and provides a list of objects in the database that match the search string. We pick `GOES-1` and use the unique id provided for this object (note that in this case we have to use `id_type='id'`):

In [4]:
eph = Ephem.from_horizons('-108366', id_type='id', epochs=[2453452.123245, 2453453.34342, 2453454.342342])
print(eph.table)

          targetname                datetime_str       ...  PABLon   PABLat 
                                                       ...   deg      deg   
----------------------------- ------------------------ ... -------- --------
GOES-1 (spacecraft) (-108366) 2005-Mar-22 14:57:28.368 ... 236.6495   7.1011
GOES-1 (spacecraft) (-108366) 2005-Mar-23 20:14:31.488 ...  97.7863 -11.5368
GOES-1 (spacecraft) (-108366) 2005-Mar-24 20:12:58.349 ...  98.5263 -12.7094


Using Ephem objects
-------------------

... please refer to [this notebook](https://github.com/NASA-Planetary-Science/sbpy-tutorial/blob/master/notebooks/data/General_concepts.ipynb) for some examples.

Computing Ephemerides with OpenOrb
-------------------------------

`sbpy.data.Ephem.from_oo` provides a way to compute ephemerides from an [sbpy.data.Orbit](https://github.com/NASA-Planetary-Science/sbpy-tutorial/blob/master/notebooks/data/General_concepts.ipynb) object. This function requires [pyoorb](https://github.com/oorb/oorb/tree/master/python) to be installed.

Due to a slight complication, it is currently not possible to run `from_oo` in a jupyter notebook. We are working on this problem.

However, you can use the following code snippet on your computer locally to compute ephemerides for a number of orbits that were obtained from the JPL Horizons system as observed from he Discovery Channel Telescope (observatory code `'G37'`):

In [5]:
from sbpy.data import Ephem, Orbit
from astropy.time import Time
from numpy import arange

orbits = Orbit.from_horizons(['12893', '3552', '2018 RC3'])
epochs = Time.now().jd + arange(0, 30, 1/24)

eph = Ephem.from_oo(orbits, epochs, location='G37')

If `pyoorb` is installed properly, this code snippet will not lead to the error message shown here.